In [102]:
import pandas as pd
import openpyxl
import sqlalchemy
from sqlalchemy import create_engine
import pymysql

In [103]:
#print(f"Using file path: '{file_path}'")

Loading & reading

In [104]:
# Load the Excel file
data = pd.read_excel('../data/raw/Global Super Store dataset.xlsx')



In [105]:
# Load the CSV file
#data = pd.read_csv('../data/raw/Global_Super_Store.csv')

In [106]:
data.shape

(51290, 24)

Check empty(s)

In [107]:
data.head
data.isnull().sum() 

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
Product ID            0
Category              0
Sub-Category          0
Product Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
dtype: int64

For my purposes, missing postal codes OK

Start Cleaning

In [108]:
# Handle missing values
# Fill missing numeric values with the mean
data['Sales'].fillna(data['Sales'].mean(), inplace=True)

#Convert Date columns to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], format='%d-%m-%Y')

data.columns.str.strip()

C:\Users\fabia\AppData\Local\Temp\ipykernel_33540\559077648.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sales'].fillna(data['Sales'].mean(), inplace=True)


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [109]:
data = data[data['Sales'] > 0]


In [110]:
#Ensure correct data types for numeric columns
data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce')
data['Profit'] = pd.to_numeric(data['Profit'], errors='coerce')
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

#Convert categorical columns to category data type
categorical_columns = ['Ship Mode', 'Customer ID', 'Segment', 'City', 'State', 'Category', 'Sub-Category', 'Product Name', 'Order Priority']
for col in categorical_columns:
    data[col] = data[col].astype('category')

#Check for consistency in categorical variables (e.g., capitalization in 'State')
data['State'] = data['State'].str.strip().str.title()

Check data shape after cleaning

In [111]:
data.shape

(51290, 24)

In [112]:
data.rename(columns={
    'Product ID': 'ProductID', 
    'Customer Name': 'FullName',
    'Order Priority': 'OrderPriority',
    'Order Date': 'OrderDate',
    'Ship Mode': 'ShippingMode',
    'Quantity': 'Quantity',
    'Segment': 'Segment',
    'Sales': 'Price',
    'Discount': 'Discount',
    'Profit': 'Profit',
    'Address ID': 'AddressID',
    'City': 'City',
    'State': 'State',
    'Country': 'Country',
    'Postal Code': 'PostalCode',
    'Region': 'Region',
    'Ship Date': 'ShippingDate',
    'Ship Cost': 'ShippingCost'
}, inplace=True)

In [113]:
display(data)

,Row ID,Order ID,OrderDate,ShippingDate,ShippingMode,Customer ID,FullName,Segment,City,State,...,ProductID,Category,Sub-Category,Product Name,Price,Quantity,Discount,Profit,Shipping Cost,OrderPriority
0,1,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,...,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium
1,2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium
2,3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium
3,4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium
4,5,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-AVE-10004570,Office Supplies,Binders,"Avery Binder Covers, Economy",11.07,1,0.0,3.42,1.98,High
51286,51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,TEC-LOG-10004419,Technology,Accessories,"Logitech Flash Drive, USB",61.44,2,0.0,18.42,13.02,High
51287,51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-BOS-10002705,Office Supplies,Art,"Boston Highlighters, Fluorescent",80.52,4,0.0,20.88,8.78,High
51288,51289,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-ENE-10004132,Office Supplies,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.0,33.84,18.97,High


Realized that 'Sales' should be 'Price' column

In [114]:
#data.rename(columns={'Sales': 'Price'}, inplace=True)

In [115]:
display(data)

,Row ID,Order ID,OrderDate,ShippingDate,ShippingMode,Customer ID,FullName,Segment,City,State,...,ProductID,Category,Sub-Category,Product Name,Price,Quantity,Discount,Profit,Shipping Cost,OrderPriority
0,1,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,...,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium
1,2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium
2,3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium
3,4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium
4,5,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-AVE-10004570,Office Supplies,Binders,"Avery Binder Covers, Economy",11.07,1,0.0,3.42,1.98,High
51286,51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,TEC-LOG-10004419,Technology,Accessories,"Logitech Flash Drive, USB",61.44,2,0.0,18.42,13.02,High
51287,51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-BOS-10002705,Office Supplies,Art,"Boston Highlighters, Fluorescent",80.52,4,0.0,20.88,8.78,High
51288,51289,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,...,OFF-ENE-10004132,Office Supplies,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.0,33.84,18.97,High


Export to CSV

In [116]:

data.to_csv('../data/clean/Global_Super_Store.csv', index=False)

In [117]:
# Check the first few rows of the DataFrame
print(data.head())

   Row ID        Order ID  OrderDate ShippingDate    ShippingMode Customer ID  \
0       1  MX-2014-143658 2014-10-02   2014-10-06  Standard Class    SC-20575   
1       2  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
2       3  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
3       4  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   
4       5  MX-2012-155047 2012-10-15   2012-10-20  Standard Class    KW-16570   

         FullName   Segment           City             State  ...  \
0    Sonia Cooley  Consumer    Mexico City  Distrito Federal  ...   
1  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
2  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
3  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
4  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   

         ProductID         Category Sub-Category  \
0  OFF-LA-10002782  Office Supplies       Labe

In [118]:
username = 'root'
password = 'q1w2e3r4'
host = '127.0.0.1'
database_name = 'globalsuperstore'

# Connection string format: 'mysql+pymysql://<username>:<password>@<host>/<database_name>'
connection_string = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

# Create an engine to connect to the MySQL database
engine = create_engine(connection_string)

Write dataframe to SQL

In [119]:
# Test
# data.to_sql('table_name', con=engine, if_exists='replace', index=False)

In [120]:
# Load CSV
df = pd.read_csv('../data/clean/Global_Super_Store.csv')


In [126]:
# Establish a connection to MySQL
connection = pymysql.connect(host='127.0.0.1',  # Change to your MySQL host
                               user='root',  # Change to your MySQL username
                               password='q1w2e3r4',  # Change to your MySQL password
                               database='globalsuperstore')  # Change to your database name


In [127]:
try:
    with connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS Shipping;")
        cursor.execute("DROP TABLE IF EXISTS Sales;")
        cursor.execute("DROP TABLE IF EXISTS Address;")
        cursor.execute("DROP TABLE IF EXISTS Customers;")
        cursor.execute("DROP TABLE IF EXISTS Products;")
        cursor.execute("DROP TABLE IF EXISTS Orders;")
        
        # Create Orders table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Orders (
                OrderID VARCHAR(100) PRIMARY KEY,
                ProductID VARCHAR(100),
                CustomerID VARCHAR(100),
                OrderDate DATE,
                Market VARCHAR(100),
                OrderPriority VARCHAR(50),
                Quantity INT,
                Segment VARCHAR(50),
                FOREIGN KEY (ProductID) REFERENCES Products(ProductID),
                FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
            );
        """)

        # Create Products table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Products (
                ProductID VARCHAR(100) PRIMARY KEY,
                ProductName VARCHAR(255),
                Category VARCHAR(100),
                Sub_category VARCHAR(100)
            );
        """)

        # Create Customers table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Customers (
                CustomerID VARCHAR(100) PRIMARY KEY,
                FullName VARCHAR(255)
            );
        """)

        # Create Shipping table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Shipping (
                OrderID VARCHAR(100),
                AddressID INT,
                ShippingDate DATE,
                ShippingMode VARCHAR(100),
                ShippingCost DECIMAL,
                PRIMARY KEY (OrderID, AddressID),
                FOREIGN KEY (OrderID) REFERENCES Orders(OrderID)
            );
        """)

        # Create Address table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Address (
                AddressID INT PRIMARY KEY,
                City VARCHAR(100),
                State VARCHAR(100),
                Country VARCHAR(100),
                PostalCode INT,
                Region VARCHAR(50)
            );
        """)

        # Create Sales table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Sales (
                Order ID VARCHAR(100),
                Sales DECIMAL,
                Discount DECIMAL,
                Profit DECIMAL,
                PRIMARY KEY (Order ID),
                FOREIGN KEY (Order ID) REFERENCES Orders(OrderID)
            );
        """)

        # Insert data into Products table
        for index, row in df.iterrows():
            product_query = """
                INSERT INTO Products (ProductID, Product Name, Category, Sub_category)
                VALUES (%s, %s, %s, %s)
            """
            cursor.execute(product_query, (row['ProductID'], row['ProductName'], row['Category'], row['Sub-category']))

        # Insert data into Customers table
        for index, row in df.iterrows():
            customer_query = """
                INSERT INTO Customers (Customer ID, FullName)
                VALUES (%s, %s)
            """
            cursor.execute(customer_query, (row['Customer ID'], row['FullName']))

        # Insert data into Orders table
        for index, row in df.iterrows():
            order_query = """
                INSERT INTO Orders (Order ID, ProductID, Customer ID, OrderDate, Market, OrderPriority, Quantity, Segment)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(order_query, (row['OrderID'], row['ProductID'], row['CustomerID'], row['OrderDate'],
                                         row['Market'], row['OrderPriority'], row['Quantity'], row['Segment']))

        # Insert data into Address table
        for index, row in df.iterrows():
            address_query = """
                INSERT INTO Address (AddressID, City, State, Country, PostalCode, Region)
                VALUES (%s, %s, %s, %s, %s, %s)
            """
            cursor.execute(address_query, (row['AddressID'], row['City'], row['State'], row['Country'],
                                           row['PostalCode'], row['Region']))

        # Insert data into Sales table
        for index, row in df.iterrows():
            sales_query = """
                INSERT INTO Sales (OrderID, Sales, Discount, Profit)
                VALUES (%s, %s, %s, %s)
            """
            cursor.execute(sales_query, (row['OrderID'], row['Sales'], row['Discount'], row['Profit']))

        # Insert data into Shipping table
        for index, row in df.iterrows():
            shipping_query = """
                INSERT INTO Shipping (OrderID, AddressID, ShippingDate, ShippingMode, ShippingCost)
                VALUES (%s, %s, %s, %s, %s)
            """
            cursor.execute(shipping_query, (row['OrderID'], row['AddressID'], row['ShippingDate'], row['ShippingMode'],
                                            row['ShippingCost']))

    # Commit the changes
    connection.commit()
    print("Data inserted successfully!")

finally:
    connection.close()

OperationalError: (3730, "Cannot drop table 'customers' referenced by a foreign key constraint 'orders_ibfk_1' on table 'orders'.")

Feature Engineering start

In [ ]:
#Create new columns for feature engineering
data['Order Duration'] = (data['Ship Date'] - data['Order Date']).dt.days
data['Profit Margin'] = data['Profit'] / data['Sales']

data.drop(columns=['Row ID'], inplace=True)